### Prediction



#### Importing Libraries

In [8]:
import dill
import numpy as np
import pandas as pd

In [9]:
model_path = r"../models/dynamic_pricing_model.pkl"
scaler_path = r"../models/standard_scaler.pkl"
with open(model_path, "rb") as f:
    model = dill.load(f)
with open(scaler_path, "rb") as f:
    scaler = dill.load(f)

In [10]:
# Function to map vehicle type to numeric value
def get_vehicle_type_numeric(vehicle_type):
    # Define a mapping from vehicle type to numeric value
    vehicle_type_mapping = {
        "Premium": 1,
        "Economy": 0
    }
    # Get the numeric value for the given vehicle type
    vehicle_type_numeric = vehicle_type_mapping.get(vehicle_type)
    return vehicle_type_numeric
  
# Predicting price using user input values
def predict_price(number_of_riders, number_of_drivers, Expected_Ride_Duration, vehicle_type):
    # Convert vehicle type to numeric representation
    vehicle_type_numeric = get_vehicle_type_numeric(vehicle_type)
    # Raise error if vehicle type is invalid
    if vehicle_type_numeric is None:
        raise ValueError("Invalid vehicle type")
    
    # Convert lists to numpy arrays for numerical computation
    number_of_riders = np.array(number_of_riders)
    number_of_drivers = np.array(number_of_drivers)
    
    # Fit a polynomial regression model to the data
    coefficients = np.polyfit(number_of_riders.ravel(), number_of_drivers.ravel(), deg=2)
    poly = np.poly1d(coefficients)
    
    # Calculate division feature
    division_feature = poly(number_of_riders / number_of_drivers)
    
    # Create input data array for prediction
    input_data = np.array([number_of_riders, number_of_drivers, Expected_Ride_Duration, vehicle_type_numeric, division_feature])
    
    # Reshape input data for compatibility with the model
    input = pd.DataFrame(input_data.reshape(1, -1))
    
    # Scale input data using scaler object
    scaled_input_data = scaler.transform(input)
    
    # Make price prediction using the model
    predicted_price = model.predict(scaled_input_data)
    return predicted_price

# Example prediction using user input values
user_number_of_riders = 42
user_number_of_drivers = 31
user_vehicle_type = "Premium"
Expected_Ride_Duration = 76
predicted_price = predict_price(user_number_of_riders, user_number_of_drivers, Expected_Ride_Duration, user_vehicle_type)

# Print the predicted price
print("Predicted price:", np.expm1(predicted_price))


Predicted price: [492.07521335]


C:\Users\sudhir\AppData\Local\Temp\ipykernel_5516\2594863263.py:25: RankWarning: Polyfit may be poorly conditioned
  coefficients = np.polyfit(number_of_riders.ravel(), number_of_drivers.ravel(), deg=2)
c:\D Drive\Projects\Dynamic Pricing Engine\.venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
